<a href="https://colab.research.google.com/github/Deekshithxyz/BigData/blob/main/IPLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [3]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [5]:
!pip install -q findspark

In [6]:
import findspark

In [7]:
findspark.init()

In [8]:
from pyspark.sql import SparkSession

In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, StringType 

In [10]:
from pyspark.sql.functions import col

In [11]:
spark = (SparkSession
.builder
.appName("IPL")
.getOrCreate())

In [12]:
schema_matches = StructType([StructField("match_id",IntegerType(),True),StructField("date",TimestampType(),True),StructField("player_of_match",StringType(),True),StructField("venue_id",IntegerType(),True),StructField("neutral_venue",IntegerType(),True),StructField("team1",StringType(),True),StructField("team2",StringType(),True),StructField("toss_winner",StringType(),True),StructField("toss_decision",StringType(),True),StructField("winner",StringType(),True),StructField("result",StringType(),True),StructField("result_margin",IntegerType(),True),StructField("eliminator",StringType(),True),StructField("method", StringType(),True),StructField("umpire1", StringType(),True),StructField("umpire2", StringType(),True)])

In [13]:
matches = spark.read.format("csv").option("header", "true").option("inferSchema", "true").schema(schema_matches).load("ipl_matches.csv")

# New Section

In [14]:
venues = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("ipl_venue.csv")

In [15]:
ball_by_ball = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("ipl_ball_by_ball.csv")

In [16]:
matches.createOrReplaceTempView("MATCH")

In [17]:
venues.createOrReplaceTempView("VENUE")

In [18]:
ball_by_ball.createOrReplaceTempView("BALL")

In [19]:
a = spark.sql("select VENUE.venue_id, VENUE.venue, VENUE.city, tvenue.count from VENUE inner join (select venue_id, count(match_id) as count from MATCH where eliminator='Y' group by venue_id order by count desc limit 3) as tvenue on VENUE.venue_id=tvenue.venue_id order by count desc")

In [20]:
b = spark.sql("select count(fielder) as catches from BALL where dismissal_kind='caught' group by fielder order by catches desc limit 1")

In [21]:
c = spark.sql("select bowler, count(*) as wickets from BALL where is_wicket=1 and match_id in (select match_id from MATCH where method='D/L') group by bowler order by wickets desc limit 1")

In [22]:
d = spark.sql("select batsman, sum(batsman_runs)*100/count(*) as strike_rate  from BALL where overs >= 7 and not extras_type='wides' and not extras_type='noballs' group by batsman order by strike_rate desc limit 1")

In [23]:
e = spark.sql("select sum(A.extra_runs) as extra_runs, first(C.venue) as venue, first(C.city) as city from BALL as A inner join MATCH as B on A.match_id=B.match_id inner join VENUE as C on B.venue_id=C.venue_id group by C.venue_id order by extra_runs desc limit 1")

In [24]:
f = spark.sql("select * from (select player_of_match, count(*) as count from MATCH where neutral_venue=1 group by player_of_match) where count=(select max(count) from (select count(*) as count from MATCH where neutral_venue=1 group by player_of_match))")

In [25]:
player_dismissed=ball_by_ball.groupby('player_dismissed').count()

In [26]:
runs=ball_by_ball.groupBy('batsman').sum('batsman_runs')

In [27]:
joined_data=player_dismissed.join(runs, player_dismissed.player_dismissed == runs.batsman, 'inner')

In [28]:
AVG=joined_data.withColumn("Batsmen Avg", col("sum(batsman_runs)")/col("count")).sort('Batsmen Avg',ascending=False)

In [29]:
g = AVG.select('batsman','Batsmen Avg')

In [30]:
h = spark.sql("select umpire, count(*) as count from ((select umpire1 as umpire from MATCH) union all (select umpire2 from MATCH)) as matches group by umpire order by count desc")

In [31]:
i = spark.sql("select B.match_id,first(C.venue),first(C.city),sum(A.batsman_runs) from BALL as A inner join MATCH as B on A.match_id=B.match_id inner join VENUE as C on B.venue_id =C.venue_id where A.batsman = 'V Kohli' group by B.match_id order by sum(A.batsman_runs) desc limit 1")

In [32]:
j = spark.sql("select sum(count) from (select winner, count(winner) as count from MATCH where toss_winner=winner group by winner)")